In [14]:
import pandas as pd
import numpy as np
import os
import json
import pprint
import matplotlib.pylab as pylab
from pylab import plot,show
import numpy as np
import cPickle as pickle
import time

%matplotlib inline

# Limit rows disp# Limit rows displayed in notebook
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 2)
pylab.rcParams['figure.figsize'] = 16, 12

Loading business details

In [15]:
pd_lasvegas = pickle.load( open('pd_lasvegas.pkl', 'rb'))
pd_lasvegas.rename(columns={'date_x': 'review_date'}, inplace=True)
pd_lasvegas.rename(columns={'date_y': 'start_date'}, inplace=True)
pd_lasvegas.rename(columns={'stars_y': 'stars'}, inplace=True)
pd_lasvegas.reset_index(level=0, inplace=True)
pd_old_business_vegas = pd_lasvegas[pd_lasvegas['new'] == False]
pd_new_business_vegas = pd_lasvegas[pd_lasvegas['new'] == True]

Creating mesh data

In [18]:
'''Calculate the mean change in the ratings for all old business given a start date of one new business'''
def calcAverage(df):
    df_before = df[df['before'] == True]
    df_after = df[df['before'] == False]

    l1 = len(df_before['stars'])
    l2 = len(df_after['stars'])

    if l1 == 0 or l2 == 0:
        return -1
    else:
        a = sum(df_before['stars']) / (len(df_before['stars'])*1.0)
        b = sum(df_after['stars']) / (len(df_after['stars'])*1.0)
        if a - b > 0:
            return -1
        else:
            return 1
    

attrs = [u'attributes_Accepts Credit Cards', u'attributes_Accepts Insurance',
       u'attributes_Ages Allowed', u'attributes_Alcohol',
       u'attributes_Ambience', u'attributes_Attire', u'attributes_BYOB',
       u'attributes_BYOB/Corkage', u'attributes_By Appointment Only',
       u'attributes_Caters', u'attributes_Coat Check',
       u'attributes_Corkage', u'attributes_Delivery',
       u'attributes_Dietary Restrictions', u'attributes_Dogs Allowed',
       u'attributes_Drive-Thru', u'attributes_Good For',
       u'attributes_Good For Dancing', u'attributes_Good For Groups',
       u'attributes_Good For Kids', u'attributes_Good for Kids',
       u'attributes_Hair Types Specialized In', u'attributes_Happy Hour',
       u'attributes_Has TV', u'attributes_Music',
       u'attributes_Noise Level', u'attributes_Open 24 Hours',
       u'attributes_Order at Counter', u'attributes_Outdoor Seating',
       u'attributes_Parking', u'attributes_Payment Types',
       u'attributes_Price Range', u'attributes_Smoking',
       u'attributes_Take-out', u'attributes_Takes Reservations',
       u'attributes_Waiter Service', u'attributes_Wheelchair Accessible',
       u'attributes_Wi-Fi', u'categories',
       u'hours_Friday', u'hours_Monday', u'hours_Saturday',
       u'hours_Sunday', u'hours_Thursday', u'hours_Tuesday',
       u'hours_Wednesday',u'neighborhoods', u'open']

def getVector(old, new):
    res = []
    for attr in attrs:
        if pd.isnull(old[attr]) or pd.isnull(new[attr]):
            res.append(-1)
        elif old[attr] == new[attr]:
            res.append(1)
        else:
            res.append(-1)
    return res

def getVectors(oldVecs, newVec):
    res = []
    for bizid, oldVec in oldVecs:
        res.append(getVector(oldVec.iloc[0], newVec))
    return res

#calculate mean for every new business
import datetime

def todate(d):
    return datetime.datetime.strptime(d, '%Y-%m-%d')

diff = datetime.timedelta(days=60)

# Use cluster 0,1 for training
X, Y = [], []

for c in [0]:
    pd_business_cluster =  pd_lasvegas[pd_lasvegas['cluster'] == c]
    pd_old_business_cluster = pd_business_cluster[pd_business_cluster['new'] == False]
    pd_old_business_stars = pd_old_business_cluster[['business_id', 'review_date', 'stars']]
    pd_new_business_cluster = pd_business_cluster[pd_business_cluster['new'] == True]
    pd_new_business_group = pd_new_business_cluster.groupby('business_id')
    
    #print "Calculating vector for cluster ", c
    start_time = time.time()
    for pd_new_business_id, pd_new_business_details in pd_new_business_group:
        print "@ ", pd_new_business_id 
        start_date = todate(pd_new_business_details.iloc[0]['start_date'])
        temp_x = getVectors(pd_old_business_cluster.groupby("business_id"), pd_new_business_details.iloc[0])
        pd_old_business_stars = pd_old_business_stars[pd_old_business_stars.review_date.apply(todate) < start_date + diff]
        pd_old_business_cluster['before'] = (pd_old_business_cluster['review_date'].apply(todate) < start_date)
        temp_y = pd_old_business_cluster.groupby(['business_id']).apply(calcAverage).values.tolist()
        X.extend(temp_x)
        Y.extend(temp_y)
        
        if len(X) > 10500:
            break
            
    end_time = time.time()
    #print "Time taken is ", end_time - start_time
    #print "Completed calculating vector for cluster ", c

@  -lRpLc285MIpDKHQk3Tqug
@  -tCiAYMimz6yQaaXiK7e6Q
@  0GCpesRgXSVluFoB2otXlA
@  0IV5L36XeQShzWiUhom9bQ
@  0vmWkXuouMhbdTakyvZZ-g
@  0w3SqrrhkxB_7t0Vhr6tQQ
@  1Vaz1OBQGj7xiCxIBuQIvA
@  1ZiGYjB9TMwmmvbJRWrB1g
@  1hY0Q3BYDq6CJDQay2xRQA
@  1onzRxmG-8J5ecUdHgkRhg
@  2eR5ncG5RA2AaREzQRVTxA
@  2lt03ZLGvgPApOKS7eKGOQ
@  2pK1ecVaqpgp1k1OFJQ9Gg
@  2tP7IwNgrcLepEj0wUh-Dw
@  3FhOLs1Sxc89loLnIAQl1g
@  3GTtkOthmbfipurFNwUZ7Q
@  3Jhosy4nKJO3oUgU2ZfMIQ
@  3_H-sx_fq0hJJ8S2W02frA
@  3g0GUgnRE3ZNVXqKXebBAg
@  3l2rx67A8ZuTnFTQrl-75Q
@  3uUYfeMGAywRy1aN01n-Sg
@  4V20KwXSSWz7A2OuiixVww
@  5CuhWcKsiqBxAt_ZHwW8Vw
@  5m2HPI3fp6Eq565mlquL0w


/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.svm import SVC

#X, Y = pickle.load(open('X_0.pkl', 'rb')), pickle.load(open('Y_0.pkl', 'rb'))
X_train, Y_train = X[:10000], Y[:10000]
X_test, Y_test = X[10000:10000+500], Y[10000:10000+500]

logistic = linear_model.LogisticRegression()
print('LogisticRegression score: %f'
      % logistic.fit(X_train, Y_train).score(X_test, Y_test))
clf = SVC()
print('SVM score: %f' % clf.fit(X_train, Y_train).score(X_test, Y_test))

LogisticRegression score: 0.598000
SVM score: 0.604000
